### Feed Water Composition

The following table presents the feed water analysis used for the ion exchange system design:

In [ ]:
# Display water composition table
feed_ions = ion_tracking.get('feed_mg_l', {})

# Create water quality dataframe with proper equivalent weight calculations
water_quality = []
for ion, conc in feed_ions.items():
    if conc and conc > 0:
        # Use mg_to_meq function from parameters if available, otherwise calculate
        if 'mg_to_meq' in locals():
            meq_l = mg_to_meq(conc, ion)
        else:
            # Fallback calculation
            meq_l = conc / 20  # Approximate

        water_quality.append({
            'Ion': ion,
            'Concentration': format_with_units(conc, 'mg/L', 2) if 'format_with_units' in locals() else f"{conc:.2f} mg/L",
            'Equivalent': format_with_units(meq_l, 'meq/L', 3) if 'format_with_units' in locals() else f"{meq_l:.3f} meq/L" if meq_l else "—"
        })

if water_quality:
    df_water = pd.DataFrame(water_quality)
    display(df_water.style.set_caption('Feed Water Composition'))
else:
    print("No feed water composition data available")

# Calculate total hardness using proper conversion factors
ca_mg_l = feed_ions.get('Ca_2+', 0)
mg_mg_l = feed_ions.get('Mg_2+', 0)
# Conversion factors to CaCO3 equivalent
ca_to_caco3 = 2.497  # 100.09/40.078
mg_to_caco3 = 4.116  # 100.09/24.305
total_hardness = ca_mg_l * ca_to_caco3 + mg_mg_l * mg_to_caco3

if total_hardness > 0:
    print(f"\nTotal Hardness: {total_hardness:.1f} mg/L as CaCO3")
    target_hardness = performance.get('effluent_hardness_mg_l_caco3')
    if target_hardness is not None:
        print(f"Target Effluent Hardness: {target_hardness:.1f} mg/L as CaCO3")

### Design Targets

The ion exchange system is designed to meet the following performance targets:

In [ ]:
# Display design targets
targets = []

# Add effluent hardness if available
eff_hardness = performance.get('effluent_hardness_mg_l_caco3')
if eff_hardness is not None:
    targets.append(['Effluent Hardness', format_with_units(eff_hardness, 'mg/L as CaCO3', 1) if 'format_with_units' in locals() else f"{eff_hardness:.1f} mg/L as CaCO3"])

# Add flow rate
flow_rate = design.get('flow_m3_hr') or design.get('flow_m3_h')
if flow_rate:
    targets.append(['Service Flow Rate', format_with_units(flow_rate, 'm³/hr', 1) if 'format_with_units' in locals() else f"{flow_rate:.1f} m³/hr"])

# Add service hours
service_hrs = performance.get('service_hours')
if service_hrs is not None:
    targets.append(['Minimum Service Run', format_with_units(service_hrs, 'hours', 1) if 'format_with_units' in locals() else f"{service_hrs:.1f} hours"])

# Add resin type
targets.append(['Resin Type', resin_info['display_name']])

# Only add alkalinity if it exists and is not None
alk_value = performance.get('effluent_alkalinity_mg_l_caco3')
if alk_value is not None:
    targets.append(['Effluent Alkalinity', format_with_units(alk_value, 'mg/L as CaCO3', 1) if 'format_with_units' in locals() else f"{alk_value:.1f} mg/L as CaCO3"])

if targets:
    df_targets = pd.DataFrame(targets, columns=['Parameter', 'Value'])
    display(df_targets.style.set_caption('Design Performance Targets'))
else:
    print("No design targets available")